# 흐름 (괄호는 코스닥만)
1. 읽어들이기 
2. 테이블 다중인덱스로 만들기 --> 편하게 하기위해서 --> PEIS 랑 똑같은 데이터 프레임형태
2. 목표주가, 당시주가, 수정주가, 애널리스트등급, 시가총액 일자로 세워서 붙이기
3. 수정주가 기준 수익률 계산해주기
4. fillna 99 해서 수익률 없는거 99해주기 --> 나중에 지우기 쉽게하기위해 임의로 박아준거임
* (코스닥) - "스팩" 데이터 제거하기 
5. 목표주가랑 현재주가 기준 컨센 구하기 
6. PEIS랑 주가데이터랑 결합해서 저장

# 수익률 PEIS점수테이블에 붙이기 

In [1]:
import pandas as pd
import numpy as np
import warnings
from Fin import *
warnings.filterwarnings(action='ignore')

# 0. 파일불러들이기 붙여질 파일(PEIS) + 붙일 파일(price)
# 목표 : 주가랑 주가상승률, 애널리스트등급 붙이기

In [2]:
PEIS_table = pd.read_excel("Data/5_4_data10.xlsx",sheet_name = "kospi_peis")
PROFIT_table = pd.read_excel("Data/5_4_data10.xlsx",sheet_name = "kospi_stock")

# 데이터 전처리 
# 1. 주가랑 주가상승률, 애널리스트등급 행렬로 일자로 만들기

In [3]:
PROFIT_table = MakeLookGood(PROFIT_table, years = list(range(10,22)))
Onetable = MakeOnetable(PROFIT_table, ['수정주가(원)','투자의견점수 (E3)(포인트)'], ['ADJprice', 'Consensus'])
Onetable['deltaprice'] = Caldeltaprice(Onetable['ADJprice'])
# 여기서 deltaprice 수정사항 넣어야함
Onetable = Onetable.fillna(99)  

In [4]:
# 1.5 컨센 점수 가공

In [5]:
consen_cut = pd.cut(Onetable['Consensus'],bins= [-0.1, 0, 2.5, 3.5, 4.5, 5], labels = [0,1,2,3,4])
Onetable['consen_cut'] = consen_cut

# 코스피도 스팩이 있음 

In [ ]:
names = Onetable['Name']
specindex = returnspecindex(names)
Onetable.drop(specindex,inplace = True)

# 2. 붙이기

In [6]:
PEIS_table = pd.concat([PEIS_table,Onetable],axis=1)

# 3. 저장하기

In [7]:
# 이렇게 만든거는.. 일단 주가 정보가 없는 친구들도 있어서 얘들은 포트폴리오로 구성하기가 힘들 듯함 상장전인듯
# 그리고 중간에 상폐당한 친구들은 또 PEIS는 있는데 주가가 없어서 포폴게산이 안될떄가 있을 거 같음 우짬.. 
PEIS_table.to_excel("Data/KospiPEISData&PriceData.xlsx")

In [104]:
PEIS_table

,Symbol,Name,회계년,총자산(천원),총자산(평균)(천원),영업이익(천원),매출원가(천원),판매비와관리비(천원),금융원가(비영업)(천원),기타비용(비영업)(천원),...,매출액(천원),매출총이익(천원),영업활동으로인한현금흐름(천원),매출채권(천원),재고자산(천원),매입채무(천원),ADJprice,Consensus,deltaprice,consen_cut
0,A005930,삼성전자,2010,1.342887e+11,1.232343e+11,1.662103e+10,1.026668e+11,3.534247e+10,7.700099e+09,1.079935e+09,...,1.546303e+11,5.196350e+10,2.382678e+10,1.915311e+10,1.336452e+10,9.148661e+09,18980.0,4.00,0.114858,3
1,A005930,삼성전자,2011,1.558003e+11,1.450445e+11,1.564429e+10,1.121451e+11,3.721236e+10,7.893421e+09,1.612690e+09,...,1.650018e+11,5.285665e+10,2.291790e+10,2.188213e+10,1.571672e+10,1.027673e+10,21160.0,4.00,0.438563,3
2,A005930,삼성전자,2012,1.810716e+11,1.684359e+11,2.904934e+10,1.266519e+11,4.540234e+10,7.934450e+09,1.576025e+09,...,2.011036e+11,7.445168e+10,3.797281e+10,2.386124e+10,1.774741e+10,9.489111e+09,30440.0,4.00,-0.098555,3
3,A005930,삼성전자,2013,2.140750e+11,1.975733e+11,3.678501e+10,1.376963e+11,5.421134e+10,7.754972e+09,1.614048e+09,...,2.286927e+11,9.099636e+10,4.670744e+10,2.498853e+10,1.913487e+10,8.437139e+09,27440.0,3.96,-0.032799,3
4,A005930,삼성전자,2014,2.304230e+11,2.222490e+11,2.502507e+10,1.282788e+11,5.290212e+10,7.294002e+09,2.259737e+09,...,2.062060e+11,7.792719e+10,3.697539e+10,2.469461e+10,1.731750e+10,7.914704e+09,26540.0,3.96,-0.050490,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13987,A950100,SBI모기지,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13988,A950100,SBI모기지,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13989,A950100,SBI모기지,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13990,A950100,SBI모기지,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 코스닥 똑같이 만들기 + 코스닥 Null값 확인해봐야함

In [81]:
PEIS_table_kosdaq = pd.read_excel("Data/5_4_data10.xlsx",sheet_name = "kosdaq_peis")
PROFIT_table_kosdaq = pd.read_excel("Data/5_4_data10.xlsx",sheet_name = "kosdaq_stock")

In [50]:
specindex = returnspecindex(names)

In [54]:
PEIS_table_kosdaq.drop(specindex,inplace = True)

NameError: name 'PEIS_table_kosdaq' is not defined

In [ ]:
PROFIT_table_kosdaq = MakeLookGood(PROFIT_table_kosdaq, years = list(range(10,22)))
# 이 아래다가 목표주가-target_price, 시가총액-MarketValue
Onetable_kosdaq = MakeOnetable(PROFIT_table_kosdaq, ['수정주가(원)','투자의견점수 (E3)(포인트)'], ['ADJprice', 'Consensus'])
Onetable_kosdaq['deltaprice'] = Caldeltaprice(Onetable_kosdaq['ADJprice'])
# 여기서 deltaprice 수정사항 넣어야함
Onetable_kosdaq = Onetable_kosdaq.fillna(99)  
consen_cut = pd.cut(Onetable_kosdaq['Consensus'],bins= [-0.1, 0, 2.5, 3.5, 4.5, 5], labels = [0,1,2,3,4])
Onetable_kosdaq['consen_cut'] = consen_cut

# 코스닥 스팩빼기

In [ ]:
names = Onetable_kosdaq['Name']
specindex = returnspecindex(names)
Onetable_kosdaq.drop(specindex,inplace = True)

# 나중에 넣을 것 예시

In [ ]:
Onetable_kosdaq = MakeOnetable(PROFIT_table_kosdaq, ['수정주가(원)','투자의견점수 (E3)(포인트)','목표주가','시가총액'], \
                               ['ADJprice', 'Consensus','TargetPrice','MarketValue'])


In [76]:
def CompuConsen(df):
    df['CompuConsen'] = 0
    Nowprice = df['총자산(천원)'] # 지금 주가
    Targetprice = df['총자산(평균)(천원)'] # 목표주가 
    for i in range(len(df)):
        if Nowprice[i]*1.4 <= Targetprice[i]:
            df['CompuConsen'][i] = 5
        elif ((Nowprice[i]*1.2 <= Targetprice[i]) | (Nowprice[i]*1.4 > Targetprice[i])) :
            df['CompuConsen'][i] = 4
        elif ((Nowprice[i]*0.8 <= Targetprice[i]) | (Nowprice[i]*1.2 > Targetprice[i])) :
            df['CompuConsen'][i] = 3
        elif (Nowprice[i]*0.8 > Targetprice[i]) :
            df['CompuConsen'][i] = 1
        else :
            df['CompuConsen'][i] = 0
            

In [77]:
CompuConsen(Onetable_kosdaq)

In [ ]:
PEIS_table_kosdaq = pd.concat([PEIS_table_kosdaq,Onetable_kosdaq],axis=1)
PEIS_table_kosdaq.to_excel("Data/KosdaqPEISData&PriceData.xlsx")